In [1]:
import speech_recognition as sr

In [2]:
import pyttsx3  
# initialize Text-to-speech engine  
engine = pyttsx3.init()  
# convert this text to speech  


In [3]:
import openai
import chronological as ch
import time
import string

openai.api_key = "" #PUT KEY HERE
model_engine = "gpt-4"

In [4]:
possible_operations = ["add_two_strings", "subtract_two_strings", "custom_function"]
operation_prompt = ""
for operation in possible_operations:
    operation_prompt += operation

In [5]:
def speech_from_microphone(prompt):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print(prompt)
        audio = r.listen(source)

    # Speech recognition from Whisper, OpenAI's speech recognition AI.
    OPENAI_API_KEY = "" #PUT KEY HERE
    try:
        speech_to_text = (r.recognize_whisper_api(audio, api_key=OPENAI_API_KEY))
        print(speech_to_text)
        return(speech_to_text)
    
    except sr.RequestError as e:
        print("Could not request results from Whisper API")


In [6]:
# This function sends prompts made by users to OpenAI's chat models, which then returns the answer.
def respond_to_prompt(user_prompt, token_limit):
    completion = openai.ChatCompletion.create(
            
        model= model_engine,
        n = 1,
        max_tokens = token_limit,
        messages=[
            {"role": "user", "content": user_prompt}
        ]
        )
    return(completion)

In [7]:
# Local text-to-speech function definition for simplicity
def text_to_speech(text_to_speech):
    text = text_to_speech  
    engine.setProperty("rate", 170)  
    engine.say(text)  
    # play the speech  
    engine.runAndWait()  

In [8]:
# Wake name and voice prompt triggers for certain functions:

wake_name = "skynet" # You can change this to anything you want

wake_name_formatting_list = [(f"\n{wake_name.capitalize()},"),(f"\n{wake_name.capitalize()}."),(f"\n{wake_name.capitalize()}!"),(f"\n{wake_name.capitalize()}?")] # Wake words 

operation_name_formatting_list = ["an operation", "a function", "run operation", "run function", "a program", "run program"] # Operation words to switch into operation mode

trigger = "FSFDFSDGGRTEGTGEGGET" # random text that wouldn't be in a prompt

sleep_words = ["sleep now", "turn off", "shut down", "power down", "power off", "go to sleep", "sleep mode", "hibernate", "i'll be back"] # Sleep words to end the program

continue_words = ["yes", "affirmative", "correct"] # Confirming words for steps that need them.

In [9]:
# Based on the function that the AI believes is the correct option to run, the corresponding function is chosen here.
def find_the_function(function_name, prompt):
    if function_name == possible_operations[0]:
        strings_to_add = respond_to_prompt(f"This is a function for adding strings. Based on the following prompt in brackets, write ONLY what you believe the two strings are, seperated by a space. ({prompt})", 30).choices[0].message.content
        add_two_strings(strings_to_add.split())
    if function_name == possible_operations[2]:
        make_a_program(prompt)

# OPERATIONS:

# Adds two strings:
def add_two_strings(strings_to_add):
    final_string = ""
    for string in strings_to_add:
        final_string += string
    print(final_string)

# Custom program creator that is used when no other premade functions are found for the needed purpose. Several steps need to be confirmed by the user as the produced code runs directly on the computer.
def make_a_program(prompt):
    text_to_speech("Write a custom program? This may use a lot of tokens.")
    check_if_proceed = speech_from_microphone("Write a custom program? This may use a lot of tokens. ")
    if any(confirm_statement in check_if_proceed.lower() for confirm_statement in continue_words):
        print(f"\n{wake_name.capitalize()}: Writing a program.")
        text_to_speech("Writing a program.")
        generated_code = respond_to_prompt(f"{prompt }Based on the prior prompt in brackets, create a python program. Do not explain anything outside of the program as your response will be run directly in python. Your response is only the interior code, do not use ` marks. Import any packages you need.", 400).choices[0].message.content
        time.sleep(1)
        print(generated_code)
        print(f"\n{wake_name.capitalize()}: The program is finished. Please check if this program is safe to run first.")
        text_to_speech("The program is finished. Please check if this program is safe to run first.")
        check_if_proceed = speech_from_microphone("\nRun program? ")
        if any(confirm_statement in check_if_proceed.lower() for confirm_statement in continue_words):
            print(f"\n{wake_name.capitalize()}: Running the program now.")
            text_to_speech("Running the program now.")
            exec(generated_code)
        else:
            return("Understood. Not running code.")
    else:
        return("Understood. Not proceeding.")

In [10]:
activated = True
while activated == True:
    sleep_timer = 0
    while True:
        if sleep_timer > 15:
            print()
            text_to_speech("Going into idle mode now.")
            input(f"\n{wake_name.capitalize()}: Press any button to wake.")
            
        voice_command = speech_from_microphone(f"\n{wake_name.capitalize()} is Listening...")
        if f"{wake_name}" in voice_command.lower():
            break
        sleep_timer += 1   
    if any(wake in voice_command.lower() for wake in wake_name_formatting_list):
        voice_command_index = voice_command.lower().find(wake_name)
        voice_command = voice_command[voice_command_index + 8:].capitalize()
    else:
        voice_command_index = voice_command.lower().find(wake_name)
        voice_command = voice_command[voice_command_index + 7:].capitalize()
    
    if any(operation in voice_command.lower() for operation in operation_name_formatting_list):
        for operation_trigger in operation_name_formatting_list:
            if operation_trigger in voice_command.lower():
                trigger = operation_trigger
                trigger_len = len(trigger)
        voice_command_operator_index = voice_command.lower().find(trigger)
        voice_command_operation_trigger_length = voice_command[trigger_len:]
        VC_temp = "Run a function" + voice_command_operation_trigger_length
        voice_command = VC_temp
        
    if "Run a function" in voice_command:
            print(f"\n{wake_name.capitalize()}: Finding your function.")
            text_to_speech("Finding your function.")
            response = respond_to_prompt(f"User Prompt: ({voice_command}). Based on the previous prompt in brackets, say only the function name that it is most likely to be (with no punctuation): {operation_prompt}. DO NOT EXPLAIN WHY. Answer: ", 15).choices[0].message.content
            print(f"\n{wake_name.capitalize()}: The function you are trying to run has been evaluated to most likely be: {response}. Is this the correct option?")
            text_to_speech(f"The function you are trying to run has been evaluated to most likely be: {response}. Is this the correct option?")
            check_if_proceed = speech_from_microphone("Listening...")
            if any(confirm_statement in check_if_proceed.lower() for confirm_statement in continue_words):
                print(f"\n{wake_name.capitalize()}: Proceeding with function {response}.")
                text_to_speech(f"Proceeding with function {response}.")
                find_the_function(response, voice_command)
    elif any(sleep_word in voice_command.lower() for sleep_word in sleep_words):
        print(f"\n{wake_name.capitalize()}: Shutting down now.")
        text_to_speech("Shutting down now.")
        activated = False
        break
    elif "voice help" in voice_command.lower():
        print(f"\n{wake_name.capitalize()}: Say the wake word, currently set to {wake_name}, to activate me. To run a function, please say create an operation or create a function, followed by the function's purpose. To confirm a command, say yes or affirmative.")
        text_to_speech(f"Say the wake word, currently set to {wake_name}, to activate me. To run a function, please say create an operation or create a function, followed by the function's purpose. To confirm a command, say yes or affirmative.")
        
    else:
        response = respond_to_prompt(f"{voice_command} You have a character limit of 300 unless specified otherwise in the sentence prior to this one.", 1000).choices[0].message.content
        print(f"\n{wake_name.capitalize()}: {response}")
        text_to_speech(response)
    time.sleep(2)


Skynet is Listening...


KeyboardInterrupt: 